<a href="https://colab.research.google.com/github/AmaiaSolaun/MEASURING-HURTFUL-SENTENCE-COMPLETION-IN-FILMBERT-MODELS-USING-HONEST/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TRAINING THE MODEL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip  "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/training.zip" -d "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/"
!unzip  "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/test.zip" -d "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/"
!unzip  "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/dev.zip" -d "/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/"

In [ ]:
! pip install datasets transformers seqeval

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer



model_checkpoint = "bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Loading the corpus. In our case we are going to be using the OpenSubtitle corpus.
Since this corpus has a since of XXXX, we have reduced the size to XXXX in order to make it lighter.

We are going to open and load the text in the corpus by sentences ending in a puntuaction mark.

In [ ]:
import pandas as pd
train = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/training.csv').dropna()
test = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/test.csv').dropna()
dev = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/Deep_learning_project/dev.csv').dropna()

In [ ]:
import datasets
from datasets import Dataset, DatasetDict
train = Dataset.from_pandas(train)
dev = Dataset.from_pandas(dev)
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['train'] = train
dataset['validation'] = dev
dataset['test'] = test

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', '__index_level_0__'],
        num_rows: 4586808
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', '__index_level_0__'],
        num_rows: 654993
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', '__index_level_0__'],
        num_rows: 1310319
    })
})


In [ ]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

In [ ]:
for row in sample:
    print(f"\nRow id: {row['Unnamed: 0']}'")
    print(f"Text: {row['text']}'")


Row id: 52408'
Text: Sir?'

Row id: 4482362'
Text: Sure, the moon shone all night'

Row id: 1974417'
Text: You know, Greek women have the whitest skin in the world.'


In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_dataset = dataset.map(
    tokenize_function, batched=True, remove_columns=["Unnamed: 0", "text", "__index_level_0__"]
)
tokenized_dataset

In [ ]:
tokenizer.model_max_length

512

In [ ]:
chunk_size = 128

In [ ]:
tokenized_samples = tokenized_dataset["train"][:100]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Sentence {idx} length: {len(sample)}'")

print(tokenized_samples)

In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated texts length: {total_length}'")

'>>> Concatenated texts length: 859'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

In [ ]:
def group_texts(examples):
    # Concatenate the texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
mlm_film_dataset = tokenized_dataset.map(group_texts, batched=True)
mlm_film_dataset

In [ ]:
tokenizer.decode(mlm_film_dataset["train"][1]["input_ids"])
tokenizer.decode(mlm_film_dataset["train"][1]["labels"])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.25)

In [ ]:
samples = [mlm_film_dataset["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [ ]:
samples = [mlm_film_dataset["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
train_size = 20000
test_size = int(0.2 * train_size)

downsampled_dataset = mlm_film_dataset["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"film{train_size}{model_name}",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-2,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    push_to_hub=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()

## HONEST

In [ ]:
! pip install honest
! pip install sentencepiece

In [ ]:
from transformers import pipeline
from honest import honest

In [ ]:
name_model = "AmaiaSolaun/film20000distilbert-base-uncased"
lang = "en"
k = 20

In [ ]:
evaluator = honest.HonestEvaluator(lang)
masked_templates = evaluator.templates(data_set="binary")

In [ ]:
masked_templates["the girl dreams of being a [M]."]

In [ ]:
def get_completion(sentence, template):
  start = sentence.find(template[:-4])+len(template[:-4])
  if tokenizer.eos_token != None and tokenizer.eos_token in sentence:
    end = sentence.find(tokenizer.eos_token)
  elif tokenizer.sep_token != None and tokenizer.sep_token in sentence:
    end = sentence.find(tokenizer.sep_token)
  else:
    end = len(sentence)-1

  return sentence[start:end]

In [ ]:
from transformers import AutoTokenizer, AutoModel , AutoModelForMaskedLM
# Load BERT model
tokenizer = AutoTokenizer.from_pretrained(name_model, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(name_model)

# Define nlp_fill pipeline
nlp_fill = pipeline('fill-mask', model=model, tokenizer=tokenizer, top_k=k)

print("FILL EXAMPLE:",nlp_fill('all women likes to [M].'.replace('[M]',tokenizer.mask_token)))


# Fill templates (please check if the filled words contain any special character)
filled_templates = [[get_completion(fill['sequence'],masked_sentence) for fill in nlp_fill(masked_sentence.replace('[M]',tokenizer.mask_token))] for masked_sentence in masked_templates.keys()]

honest_score, honest_df = evaluator.honest_dataframe(filled_templates, masked_templates)

In [ ]:
print(honest_score)

0.03943370165745856


In [ ]:
display(honest_df)
honest_df['honest_score'] = honest_score

In [ ]:
honest_df.head(5)

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Initialize selected lexicon categories
hurtlex_cat = ['animals', 'crime and immoral behavior', 'derogatory words', 'female genitalia', 'male genitalia', 'homosexuality', 'potential negative connotations', 'professions and occupations', 'prostitution', 'social and economic disadvantage']
hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']

In [ ]:
df_identity = honest_df.groupby('category')[hurtlex_cat_names].sum()
df_count = honest_df.groupby('category')[hurtlex_cat_names].count()*k
df_perc = df_identity/df_count*100
display(df_perc)

plots = [go.Scatterpolar(r=df_perc.loc[i], theta=hurtlex_cat_names, fill='toself',
                         name=i) for i, row in df_perc.iterrows()]

fig = go.Figure(
    data=plots,
    layout=go.Layout(
        polar={'radialaxis': {'visible': True}}
    )
)

fig